# Getting Started with Julia in Colab/Jupyter
You can either run this notebook in Google Colab, or using Jupyter on your own machine.

## Running on Google Colab
1. Work on a copy of this notebook: _File_ > _Save a copy in Drive_ (you will need a Google account). Alternatively, you can download the notebook using _File_ > _Download .ipynb_, then upload it to [Colab](https://colab.research.google.com/).
2. Execute the following cell (click on it and press Ctrl+Enter) to install Julia, IJulia (the Jupyter kernel for Julia) and other packages. You can update `JULIA_VERSION` and the other parameters, if you know what you're doing. Installation takes 2-3 minutes.
3. Reload this page (press Ctrl+R, or ⌘+R, or the F5 key) and continue to the _Checking the Installation_ section.

* _Note_: If your Colab Runtime gets reset (e.g., due to inactivity), repeat steps 2 and 3.

In [ ]:
%%shell
set -e

#---------------------------------------------------#
JULIA_VERSION="1.6.0" # any version ≥ 0.7.0
JULIA_PACKAGES="IJulia DataFrames CSV Pipe DataFramesMeta"
JULIA_PACKAGES_IF_GPU="CUDA"
JULIA_NUM_THREADS=4
#---------------------------------------------------#

if [ -n "$COLAB_GPU" ] && [ -z `which julia` ]; then
  # Install Julia
  JULIA_VER=`cut -d '.' -f -2 <<< "$JULIA_VERSION"`
  echo "Installing Julia $JULIA_VERSION on the current Colab Runtime..."
  BASE_URL="https://julialang-s3.julialang.org/bin/linux/x64"
  URL="$BASE_URL/$JULIA_VER/julia-$JULIA_VERSION-linux-x86_64.tar.gz"
  wget -nv $URL -O /tmp/julia.tar.gz # -nv means "not verbose"
  tar -x -f /tmp/julia.tar.gz -C /usr/local --strip-components 1
  rm /tmp/julia.tar.gz

  # Install Packages
  if [ "$COLAB_GPU" = "1" ]; then
      JULIA_PACKAGES="$JULIA_PACKAGES $JULIA_PACKAGES_IF_GPU"
  fi
  for PKG in `echo $JULIA_PACKAGES`; do
    echo "Installing Julia package $PKG..."
    julia -e 'using Pkg; pkg"add '$PKG'; precompile;"' &> /dev/null
  done

  # Install kernel and rename it to "julia"
  echo "Installing IJulia kernel..."
  julia -e 'using IJulia; IJulia.installkernel("julia", env=Dict(
      "JULIA_NUM_THREADS"=>"'"$JULIA_NUM_THREADS"'"))'
  KERNEL_DIR=`julia -e "using IJulia; print(IJulia.kerneldir())"`
  KERNEL_NAME=`ls -d "$KERNEL_DIR"/julia*`
  mv -f $KERNEL_NAME "$KERNEL_DIR"/julia  

  echo ''
  echo "Successfully installed `julia -v`!"
  echo "Please reload this page (press Ctrl+R, ⌘+R, or the F5 key) then"
  echo "jump to the 'Checking the Installation' section."
fi

Installing Julia 1.6.0 on the current Colab Runtime...
2022-01-29 12:03:41 URL:https://storage.googleapis.com/julialang2/bin/linux/x64/1.6/julia-1.6.0-linux-x86_64.tar.gz [112838927/112838927] -> "/tmp/julia.tar.gz" [1]
Installing Julia package IJulia...
Installing Julia package DataFrames...
Installing Julia package CSV...
Installing Julia package Pipe...
Installing Julia package DataFramesMeta...
Installing IJulia kernel...
[ Info: Installing julia kernelspec in /root/.local/share/jupyter/kernels/julia-1.6

Please reload this page (press Ctrl+R, ⌘+R, or the F5 key) then
jump to the 'Checking the Installation' section.


## Checking the Installation
The `versioninfo()` function should print your Julia version and some other info about the system (if you ever ask for help or file an issue about Julia, you should always provide this information).

In [1]:
versioninfo()

Julia Version 1.6.0
Commit f9720dc2eb (2021-03-24 12:55 UTC)
Platform Info:
  OS: Linux (x86_64-pc-linux-gnu)
  CPU: Intel(R) Xeon(R) CPU @ 2.20GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-11.0.1 (ORCJIT, broadwell)
Environment:
  JULIA_NUM_THREADS = 4


# Step 1. Import the necessary libraries

In [2]:
using DataFrames
using CSV
using Pipe
using DataFramesMeta

# Step 2. Import the dataset from this [address](https://raw.githubusercontent.com/guipsamora/pandas_exercises/master/04_Apply/Students_Alcohol_Consumption/student-mat.csv).


In [3]:
URL = "https://raw.githubusercontent.com/guipsamora/pandas_exercises/master/04_Apply/Students_Alcohol_Consumption/student-mat.csv"
download(URL,"student-mat.csv")

"student-mat.csv"

# Step 3. Assign it to a variable called df.

In [4]:
df = CSV.read("student-mat.csv", DataFrame)

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob
,String3,String1,Int64,String1,String3,String1,Int64,Int64,String15,String15
1,GP,F,18,U,GT3,A,4,4,at_home,teacher
2,GP,F,17,U,GT3,T,1,1,at_home,other
3,GP,F,15,U,LE3,T,1,1,at_home,other
4,GP,F,15,U,GT3,T,4,2,health,services
5,GP,F,16,U,GT3,T,3,3,other,other
6,GP,M,16,U,LE3,T,4,3,services,other
7,GP,M,16,U,LE3,T,2,2,other,other
8,GP,F,17,U,GT3,A,4,4,other,teacher
9,GP,M,15,U,LE3,A,3,2,services,other


# Step 4. For the purpose of this exercise slice the dataframe from 'school' until the 'guardian' column


In [11]:
stud_alcohol = select(df, Between("school", "guardian"))

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob
,String3,String1,Int64,String1,String3,String1,Int64,Int64,String15,String15
1,GP,F,18,U,GT3,A,4,4,at_home,teacher
2,GP,F,17,U,GT3,T,1,1,at_home,other
3,GP,F,15,U,LE3,T,1,1,at_home,other
4,GP,F,15,U,GT3,T,4,2,health,services
5,GP,F,16,U,GT3,T,3,3,other,other
6,GP,M,16,U,LE3,T,4,3,services,other
7,GP,M,16,U,LE3,T,2,2,other,other
8,GP,F,17,U,GT3,A,4,4,other,teacher
9,GP,M,15,U,LE3,A,3,2,services,other


# Step 5. Create a lambda function that will capitalize strings.

In [12]:
# Julia has an inbuilt function to do this so we won't make it
uppercasefirst("school")

"School"

# Step 6. Capitalize both Mjob and Fjob

In [ ]:
stud_alcohol[!, :Mjob] = uppercasefirst.(stud_alcohol.Mjob)
stud_alcohol[!, :Fjob] = uppercasefirst.(stud_alcohol.Fjob)

In [17]:
first(stud_alcohol,5)

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob
,String3,String1,Int64,String1,String3,String1,Int64,Int64,String,String
1,GP,F,18,U,GT3,A,4,4,At_home,Teacher
2,GP,F,17,U,GT3,T,1,1,At_home,Other
3,GP,F,15,U,LE3,T,1,1,At_home,Other
4,GP,F,15,U,GT3,T,4,2,Health,Services
5,GP,F,16,U,GT3,T,3,3,Other,Other


# Step 7. Print the last elements of the data set.

In [18]:
last(stud_alcohol, 5)

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob
,String3,String1,Int64,String1,String3,String1,Int64,Int64,String,String
1,MS,M,20,U,LE3,A,2,2,Services,Services
2,MS,M,17,U,LE3,T,3,1,Services,Services
3,MS,M,21,R,GT3,T,1,1,Other,Other
4,MS,M,18,R,LE3,T,3,2,Services,Other
5,MS,M,19,U,LE3,T,1,1,Other,At_home


# Step 9. Create a function called majority that returns a boolean value to a new column called legal_drinker (Consider majority as older than 17 years old)

In [ ]:
majority(x) = x > 17
stud_alcohol[!, :legal_drinker] = majority.(stud_alcohol.age)

In [22]:
stud_alcohol

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob
,String3,String1,Int64,String1,String3,String1,Int64,Int64,String,String
1,GP,F,18,U,GT3,A,4,4,At_home,Teacher
2,GP,F,17,U,GT3,T,1,1,At_home,Other
3,GP,F,15,U,LE3,T,1,1,At_home,Other
4,GP,F,15,U,GT3,T,4,2,Health,Services
5,GP,F,16,U,GT3,T,3,3,Other,Other
6,GP,M,16,U,LE3,T,4,3,Services,Other
7,GP,M,16,U,LE3,T,2,2,Other,Other
8,GP,F,17,U,GT3,A,4,4,Other,Teacher
9,GP,M,15,U,LE3,A,3,2,Services,Other


# Step 10. Multiply every number of the dataset by 10.

In [26]:
by10(x) = 10x
stud_alcohol[!, names(stud_alcohol, Number)] = by10.(stud_alcohol[!, names(stud_alcohol, Number)])

,age,Medu,Fedu,legal_drinker
,Int64,Int64,Int64,Int64
1,180,40,40,10
2,170,10,10,0
3,150,10,10,0
4,150,40,20,0
5,160,30,30,0
6,160,40,30,0
7,160,20,20,0
8,170,40,40,0
9,150,30,20,0


In [27]:
stud_alcohol

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob
,String3,String1,Int64,String1,String3,String1,Int64,Int64,String,String
1,GP,F,180,U,GT3,A,40,40,At_home,Teacher
2,GP,F,170,U,GT3,T,10,10,At_home,Other
3,GP,F,150,U,LE3,T,10,10,At_home,Other
4,GP,F,150,U,GT3,T,40,20,Health,Services
5,GP,F,160,U,GT3,T,30,30,Other,Other
6,GP,M,160,U,LE3,T,40,30,Services,Other
7,GP,M,160,U,LE3,T,20,20,Other,Other
8,GP,F,170,U,GT3,A,40,40,Other,Teacher
9,GP,M,150,U,LE3,A,30,20,Services,Other
